link for outline
<a id='acquire_random_data'></a>

# What are we trying to do

Acquire data for random songs from spotify API to use in comparison to songs from specific playlist

### Issues

- According to [here](https://perryjanssen.medium.com/getting-random-tracks-using-the-spotify-api-61889b0c0c27), API returns more "famous" artists/tracks at top of searches.  Luckily, can use an off-set to counteract

- Also according to above link, API results are also different according to "market" in which IP number posting `REQUEST` is located, which is co-extensive but not congruous with "country". So, should also randomize market. 

### External resources

Here's something someone made with the spotipy package that:

- randomly chooses a letter

- randomly chooses whether it's at the start, beginning, or end of a word

- randomly chooses whether it's in a song title, album, etc (set of choices here is selectable)

https://github.com/michimalek/spotipy-random

In [9]:
from spotipy import Spotify
from spotipy_random import get_random

from spotipy.oauth2 import SpotifyClientCredentials

In [15]:
auth_manager = SpotifyClientCredentials(
    client_id="1c2e33a8bebd41fbbb8a1ecf0e8c4273",
    client_secret="4964098dcc7b41f99c4178e6403645c1"
)

sp = Spotify(auth_manager=auth_manager)

# spotify_client = Spotify(auth_manager=SpotifyClientCredentials(
#                                  client_id="1c2e33a8bebd41fbbb8a1ecf0e8c4273",
#                                  client_secret="4964098dcc7b41f99c4178e6403645c1")
#                         )

random_pop_song_json: str = get_random(spotify=sp, type="track", genre="pop")